# General approach to kNN

1. Collect: any method
2. Prepare: numeric values are need for a distance calculation. A structured data format is best.
3. Anayze: any method
4. Train: does not apply to the kNN algorithm
5. Test: calculate the error rate
6. Use: This application needs to get some input data and output structure numeric values. Next, the application run the kNN algorithm on this input data and determines whitch class the input data should belong to.

## 1. Collect: 
Tengo el fichero train.csv, test.csv y gender_submission.csv (este último para ver el formato del fichero que se envía).

## 2. Prepare
Hay que preparar los datos primero del fichero de training y luego del de test.
* Extraer las labels.
* Transformar los valores no numericos en valores numéricos.
* Tratar los valores nulos.
* Establecer valores entre 0 y 1

Por simplificar obviaré el control de errores

In [45]:
import re

MAX_CLASS = 3 # Max value of passae Class
MAX_AGE = 100
MAX_SIBLINGS = 10
MAX_PARENTS = 10
MAX_TICKET = 3101317.0
MAX_FARE = 512.3292

def removeName(line):
    return re.sub('"(.*?)",','', line)

def normalizeClass(classNumber):
    return (float(classNumber) - 1.0)/ (MAX_CLASS -1)

def normalizeSex(sex):
    sexNormalizedValues = { "male" : 0.5, "female" : 1}
    
    if sex in sexNormalizedValues:
        return sexNormalizedValues[sex]
    else:
        return 0

def normalizeAge(age):
    if not age:
        return 0
    else:
        return float(age) / MAX_AGE
    
def normalizeNumberOfSiblings(siblings):
    if not siblings:
        return 0
    else:
        return float(siblings) / MAX_SIBLINGS

def normalizeNumberOfParents(parents):
    if not parents:
        return 0
    else:
        return float(parents) / MAX_PARENTS
    
def normalizePrefixTicket(ticket):
    prefixes = { 'A/5' : 0.01, 'A/5.' : 0.01, 'A./5.': 0.01, 'A.5.' : 0.01, 'PC' : 0.02, 'STON/O2.' : 0.03, 'PP' : 0.04, 'C.A.' : 0.05, 'CA' : 0.05 , 'CA.' : 0.05, 'SC/Paris' : 0.06, 'SC/PARIS' : 0.06, 'S.C./PARIS' : 0.06, 'S.C./A.4.' : 0.07, 'A/4.' : 0.08, 'A4.' : 0.08, 'A/4' : 0.08, 'S.P.' : 0.09, 'S.O.C.' : 0.10, 'SO/C' : 0.10, 'W./C.' : 0.11, 'W/C' : 0.11, 'SOTON/OQ' : 0.12, 'SOTON/O.Q.' : 0.12, 'W.E.P.' : 0.13, 'WE/P' : 0.13, 'STON/O' : 0.14, 'C' : 0.15, 'S.O.P.' : 0.16, 'Fa' : 0.17, 'F.C.C.' : 0.18, 'SW/PP' : 0.19, 'S.W./PP' : 0.19, 'SCO/W' : 0.20, 'P/PP' : 0.21, 'SC': 0.22, 'SC/AH' : 0.23, 'A/S': 0.24, 'S.O./P.P.' : 0.25, 'F.C.' : 0.26, 'SOTON/O2' : 0.27, 'C.A./SOTON' : 0.28 }

    ticketSplitedData = ticket.split(' ')
    if len(ticketSplitedData) > 1:
        ticketPrefix = ticketSplitedData[0]
        if ticketPrefix in prefixes:
            return prefixes[ticketPrefix]
        else:
            return 0
    else:
        return 0

def normalizeTicketNumber(ticket):
    return getTicketNumber(ticket) / MAX_TICKET

def getTicketNumber(ticket):
    ticketSplitedData = ticket.split(' ')
    
    if len(ticketSplitedData) == 1:
        if ticket == 'LINE':
            return 0
        else:
            return float(ticket)
    else:
        return float(ticketSplitedData[len(ticketSplitedData)-1]) 
    
def normalizeFare(fare):
    if len(fare) == 0:
        return 0
    else:
        return float(fare) / MAX_FARE
    
def normalizeCabinPrefix(cabin):
    prefixes = { 'A' : 0.1, 'B' : 0.2, 'C': 0.3, 'D' : 0.4, 'E' : 0.5, 'F' : 0.6, 'G' : 0.7 }
    
    if len(cabin) == 0:
        return 0
    else:
        cabinSplitedData = cabin.split(' ')
    
        if len(cabinSplitedData) == 1:
            return prefixes[cabin[0]]
        else:
            return prefixes[cabinSplitedData[0][0]]
    
def processTrainingFile(filename):
    file = open(filename)
    
    labels = []
    trainingData = []
    
    ticketPrefixes = []
    
    # Leer fichero linea a linea
    header = True
    for line in file.readlines():
        if not header:
            line = removeName(line)
 
            lineValueList = line.strip().split(',')
            labels.append(lineValueList[1])
            
            treatedData = []
            treatedData.append(normalizeClass(lineValueList[2]))
            treatedData.append(normalizeSex(lineValueList[3]))
            treatedData.append(normalizeAge(lineValueList[4]))
            treatedData.append(normalizeNumberOfSiblings(lineValueList[5]))
            treatedData.append(normalizeNumberOfParents(lineValueList[6]))
            treatedData.append(normalizePrefixTicket(lineValueList[7]))
            treatedData.append(normalizeTicketNumber(lineValueList[7]))
            treatedData.append(normalizeFare(lineValueList[8]))
            treatedData.append(normalizeCabinPrefix(lineValueList[9]))
                                
            print(treatedData)
            trainingData.append(treatedData)
        else:
            header = False
            
    # Guardar el fichero con las transformaciones
    # Guardar el fichero con las labels

In [46]:
processTrainingFile('titanic/train.csv')

[1.0, 0.5, 0.22, 0.1, 0.0, 0.01, 0.006826454696504743, 0.014151057562208049, 0]
[0.0, 1, 0.38, 0.1, 0.0, 0.02, 0.005674685947937602, 0.13913573538264068, 0.3]
[1.0, 1, 0.26, 0.0, 0.0, 0.03, 0.9999887144719485, 0.015468569817999833, 0]
[0.0, 1, 0.35, 0.1, 0.0, 0, 0.03669505568118319, 0.10364429745562033, 0.3]
[1.0, 0.5, 0.35, 0.0, 0.0, 0, 0.1204165843091822, 0.015712553569072387, 0]
[1.0, 0.5, 0, 0.0, 0.0, 0, 0.106689190431033, 0.01650950209357577, 0]
[0.0, 0.5, 0.54, 0.0, 0.0, 0, 0.005630833610366177, 0.10122885832000206, 0.5]
[1.0, 0.5, 0.02, 0.3, 0.1, 0, 0.11282593814176364, 0.04113566043083236, 0]
[1.0, 1, 0.27, 0.0, 0.2, 0, 0.11212720273354836, 0.021730754366528396, 0]
[0.5, 1, 0.14, 0.1, 0.0, 0, 0.07665646562412033, 0.058694292654020104, 0]
[1.0, 1, 0.04, 0.1, 0.1, 0.04, 0.0030790144960995604, 0.03259622914329302, 0.7]
[0.0, 1, 0.58, 0.0, 0.0, 0, 0.03668860680801092, 0.051822148727810165, 0.3]
[1.0, 0.5, 0.2, 0.0, 0.0, 0.01, 0.0006935763096774693, 0.015712553569072387, 0]
[1.0, 0.

KeyError: 'T'